# Metaprogramming

### Metaprogramming (メタプログラミング)
<center>
    Sympy 計算式　->　code 生成<br>
    ↓
    <br>
    <u>論理から code を生成する事</u>
    <br>
    ||
    <br>
    <b>Metaprogramming</b>
</center>

- **自分で code を書かなくて良い**

    - *バグなどの混入問題が無い*
    - *自動化できる*


- **複雑な式でも取り扱える** 

In [1]:
import sympy as sp
import numpy as np

In [2]:
sp.init_printing()

In [3]:
ux, uy, uz = sp.symbols("ux, uy, uz")
ex, ey, ez = sp.symbols("ex, ey, ez")

In [4]:
sp.expand((ex - ux) ** 2 * (ey - uy) ** 1 * (ez -uz) ** 0)

In [5]:
exp = sp.expand((ex - ux) ** 2 * (ey - uy) ** 1 * (ez -uz) ** 0)

In [6]:
# 定数を抽出 : ex, ey, ez それぞれの n の 0 乗 = 定数
exp.coeff(ex, n=0).coeff(ey, n=0).coeff(ez, n=0)

In [10]:
k_txt = "k000,k100,k010,k001,k110,k101,k011,k200,k020,k002,k120,\
k102,k210,k201,k012,k021,k111,k220,k202,k022,k211,k121,\
k112,k122,k212,k221,k222"

In [14]:
k_txt

'k000,k100,k010,k001,k110,k101,k011,k200,k020,k002,k120,k102,k210,k201,k012,k021,k111,k220,k202,k022,k211,k121,k112,k122,k212,k221,k222'

In [15]:
k_list = k_txt.split(",")
k_list

['k000',
 'k100',
 'k010',
 'k001',
 'k110',
 'k101',
 'k011',
 'k200',
 'k020',
 'k002',
 'k120',
 'k102',
 'k210',
 'k201',
 'k012',
 'k021',
 'k111',
 'k220',
 'k202',
 'k022',
 'k211',
 'k121',
 'k112',
 'k122',
 'k212',
 'k221',
 'k222']

In [17]:
kmnp = {}
N_mtx = sp.zeros(27, 27)
for row_idx, k_idx in  enumerate(k_list):
    
    # K000 ( k,m,n,p の並びになっている index順に取得 )
    m = int(k_idx[1])
    n = int(k_idx[2])
    p = int(k_idx[3])
    k_expand = (ex -ux) ** m * (ey - uy) ** n * (ez - uz) ** p
    
    # simplify と expand は set で使用する
    k_expand = sp.simplify(sp.expand(k_expand))

    # 順番に係数を抽出していく
    for col_idx, k_idx2 in enumerate(k_list):
        m2 = int(k_idx2[1])
        n2 = int(k_idx2[2])
        p2 = int(k_idx2[3])
        val = k_expand.coeff(ex, n=m2).coeff(ey, n=n2).coeff(ez, n=p2)
        N_mtx[row_idx, col_idx] = val

In [18]:
N_mtx

⎡     1             0              0              0             0            0
⎢                                                                             
⎢    -ux            1              0              0             0            0
⎢                                                                             
⎢    -uy            0              1              0             0            0
⎢                                                                             
⎢    -uz            0              0              1             0            0
⎢                                                                             
⎢   ux⋅uy          -uy            -ux             0             1            0
⎢                                                                             
⎢   ux⋅uz          -uz             0             -ux            0            1
⎢                                                                             
⎢   uy⋅uz           0             -uz            -uy

In [19]:
# 逆行列
N_mtx.inv()

⎡     1            0             0             0             0            0   
⎢                                                                             
⎢    ux            1             0             0             0            0   
⎢                                                                             
⎢    uy            0             1             0             0            0   
⎢                                                                             
⎢    uz            0             0             1             0            0   
⎢                                                                             
⎢   ux⋅uy          uy            ux            0             1            0   
⎢                                                                             
⎢   ux⋅uz          uz            0             ux            0            1   
⎢                                                                             
⎢   uy⋅uz          0             uz            uy   

## 上記を　C言語にする

In [23]:
# vector 生成
k = sp.Matrix(sp.MatrixSymbol("k", 27, 1))

In [24]:
# 行列と Matrix(vector) の積
k_tilde = N_mtx * k

In [25]:
k_tilde

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [26]:
for i in range(27):
    code = sp.ccode(k_tilde[i], assign_to=f"k_tilde : [{i}]")
    print(code)

k_tilde : [0] = k[0];
k_tilde : [1] = -ux*k[0] + k[1];
k_tilde : [2] = -uy*k[0] + k[2];
k_tilde : [3] = -uz*k[0] + k[3];
k_tilde : [4] = ux*uy*k[0] - ux*k[2] - uy*k[1] + k[4];
k_tilde : [5] = ux*uz*k[0] - ux*k[3] - uz*k[1] + k[5];
k_tilde : [6] = uy*uz*k[0] - uy*k[3] - uz*k[2] + k[6];
k_tilde : [7] = pow(ux, 2)*k[0] - 2*ux*k[1] + k[7];
k_tilde : [8] = pow(uy, 2)*k[0] - 2*uy*k[2] + k[8];
k_tilde : [9] = pow(uz, 2)*k[0] - 2*uz*k[3] + k[9];
k_tilde : [10] = -ux*pow(uy, 2)*k[0] + 2*ux*uy*k[2] - ux*k[8] + pow(uy, 2)*k[1] - 2*uy*k[4] + k[10];
k_tilde : [11] = -ux*pow(uz, 2)*k[0] + 2*ux*uz*k[3] - ux*k[9] + pow(uz, 2)*k[1] - 2*uz*k[5] + k[11];
k_tilde : [12] = -pow(ux, 2)*uy*k[0] + pow(ux, 2)*k[2] + 2*ux*uy*k[1] - 2*ux*k[4] - uy*k[7] + k[12];
k_tilde : [13] = -pow(ux, 2)*uz*k[0] + pow(ux, 2)*k[3] + 2*ux*uz*k[1] - 2*ux*k[5] - uz*k[7] + k[13];
k_tilde : [14] = -uy*pow(uz, 2)*k[0] + 2*uy*uz*k[3] - uy*k[9] + pow(uz, 2)*k[2] - 2*uz*k[6] + k[14];
k_tilde : [15] = -pow(uy, 2)*uz*k[0] + pow(uy, 2)*k[3